In [1]:
import pandas as pd
import numpy as np
from gensim.models import FastText
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.initializers import Constant
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.utils import resample
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.utils import plot_model

In [2]:
import re

def preprocess_sentence(sentence, stopwords=None):
    # 개행자 삭제
    sentence = re.sub(r'[\n\r]', ' ', sentence)
    
    # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
    # 예를 들어서 "I am a student." => "I am a student ."와 같이
    # student와 온점 사이에 거리를 만듭니다. 
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # (a-z, A-Z,가-힣,0-9, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
    sentence = re.sub(r"[^a-zA-Z가-힣0-9\.\?\!,]", " ", sentence)
    sentence = sentence.strip()

    # '키키'와 같이 연속된 키를 제거합니다.
    sentence = re.sub(r'키{2,}', '', sentence)
    sentence = re.sub(r'\b키\b', '', sentence)

    if stopwords:
        words = sentence.split()
        filtered_words = [word for word in words if word not in stopwords]
        sentence = ' '.join(filtered_words)

    return sentence

In [3]:
# data
base_dir = '../data/'

train = pd.read_csv(base_dir + 'custom_train_2.csv')
test = pd.read_csv(base_dir + 'custom_test.csv')
test['conversation'] = test['conversation'].apply(preprocess_sentence)

In [4]:
all_texts = train['conversation'].tolist()

In [5]:
# We train FastText model on all_texts
fasttext_model = FastText(
    sentences=all_texts,
    vector_size=300,
    window=5,
    min_count=2,
    sg=1,
    word_ngrams=1,
    workers=4)

# Define Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
word_index = tokenizer.word_index



# EMBEDDING_DIM = 300  # this should match the dimensionality of the FastText vectors

# # Prepare embedding matrix
# embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
# for word, i in word_index.items():
#     if word in fasttext_model.wv:
#         embedding_vector = fasttext_model.wv[word]
#         embedding_matrix[i] = embedding_vector

# # Load this into an Embedding layer
# # Note that we set trainable=False to prevent the weights from being updated during training.

# embedding_layer = Embedding(len(word_index) + 1,
#                             EMBEDDING_DIM,
#                             embeddings_initializer=Constant(embedding_matrix),
#                             trainable=False)

In [6]:
def get_vector(model, texts):
    vectors = []
    for text in texts:
        text_vectors = []
        for word in text:
            if word in model.wv.key_to_index:
                text_vectors.append(model.wv.get_vector(word))
        if len(text_vectors) > 0:
            text_vector = np.mean(text_vectors, axis=0)
            vectors.append(text_vector)
        else:
            vectors.append(np.zeros(model.vector_size))
    return np.array(vectors)

In [7]:
X_train = get_vector(fasttext_model, train['conversation'])
Y_train = tf.keras.utils.to_categorical(train['label'])
X_test = get_vector(fasttext_model, test['conversation'])

In [8]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=123)

# Model Define

In [9]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

In [10]:
X_train.shape

(7920, 1, 300)

In [11]:
# import optuna
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
!pip install wandb==0.16.0

In [13]:
import wandb
from wandb.keras import WandbCallback

wandb.login(key = 'e32bb98df2b2741a337ed3235cc4da224748f384')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [14]:
sweep_config = {
    "name": "fasttext_gesim_6",
    "metric": {"name": "val_loss", "goal": "minimize"},
    'method': 'bayes',  # 베이지안 방식으로 하이퍼파라미터 탐색
    'parameters': {
        "units" : {
            "values" : [32, 64, 128]
        },
        "learning_rate" : {
            "values" : [0.002, 0.001, 0.003, 0.005]
        },
        "dropout_rate" : {
            "values": [0.2, 0.3]
        },
        "batch_size" : {
            "values" : [16, 32,]
        }
    }
}


In [15]:
def train():
    default_config = {
        "units" : 128,
        "learning_rate" : 0.005,
        "dropout_rate" : 0.3,
        "batch_size" : 32,
    }
    
    import os

    # 환경 변수 설정
    os.environ['WANDB_AGENT_DISABLE_FLAPPING'] = 'true'

    wandb.init(config = sweep_config)
    config = wandb.config


    # Define the model architecture
    model = Sequential()
    model.add(LSTM(config.units, return_sequences=True, input_shape=(1, X_train.shape[2])))
    model.add(Dropout(config.dropout_rate))
    model.add(LSTM(config.units))
    model.add(Dropout(config.dropout_rate))
    model.add(Dense(Y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=config.learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping
    es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5, min_delta=0.0001)

    # Train the model
    model.fit(X_train,
              Y_train,
              epochs=50,
              batch_size=32,
              validation_data=(X_val, Y_val),
              callbacks=[es, WandbCallback()]
             )

    val_accuracy = model.evaluate(X_val, Y_val, verbose=0)[1]
    
    # wandb.log 함수 안에 기록하고 싶은 정보를 담습니다.
    
    wandb.log({"Validation Accuracy Rate: " : round(val_accuracy * 100, 2),
               "Validation Error Rate: " : round((1 - val_accuracy) * 100, 2)})
    
    

In [16]:
# entity와 project에 본인의 아이디와 프로젝트명을 입력하세요

sweep_id = wandb.sweep(sweep_config,
                       entity = 'devdchyeon',
                       project = 'DLTHON')

# run the sweep
wandb.agent(sweep_id,
            function=train,
            count=10)

Create sweep with ID: k94x0hcc
Sweep URL: https://wandb.ai/devdchyeon/DLTHON/sweeps/k94x0hcc


wandb: Agent Starting Run: 4hkmh0p7 with config:
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0.3
wandb: 	learning_rate: 0.003
wandb: 	units: 32
wandb: Currently logged in as: dev-dc-hyeon (devdchyeon). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/50
248/248 [==============================] - 32s 10ms/step - loss: 1.3847 - accuracy: 0.3949 - val_loss: 0.9965 - val_accuracy: 0.6131


INFO:tensorflow:Assets written to: /aiffel/aiffel/workspace/16_dlthon_1/fasttext_lstm/wandb/run-20240627_003259-4hkmh0p7/files/model-best/assets


...

Epoch 14/50
248/248 [==============================] - 1s 5ms/step - loss: 0.6655 - accuracy: 0.7564 - val_loss: 0.6485 - val_accuracy: 0.7596


INFO:tensorflow:Assets written to: /aiffel/aiffel/workspace/16_dlthon_1/fasttext_lstm/wandb/run-20240627_005137-wwhgmkj9/files/model-best/assets


INFO:tensorflow:Assets written to: /aiffel/aiffel/workspace/16_dlthon_1/fasttext_lstm/wandb/run-20240627_005137-wwhgmkj9/files/model-best/assets
wandb: Adding directory to artifact (/aiffel/aiffel/workspace/16_dlthon_1/fasttext_lstm/wandb/run-20240627_005137-wwhgmkj9/files/model-best)... Done. 0.0s


Epoch 15/50
248/248 [==============================] - 1s 5ms/step - loss: 0.6600 - accuracy: 0.7543 - val_loss: 0.6847 - val_accuracy: 0.7414
Epoch 16/50
248/248 [==============================] - 1s 5ms/step - loss: 0.6544 - accuracy: 0.7591 - val_loss: 0.6541 - val_accuracy: 0.7556
Epoch 17/50
248/248 [==============================] - 1s 5ms/step - loss: 0.6525 - accuracy: 0.7598 - val_loss: 0.6833 - val_accuracy: 0.7525
Epoch 18/50
248/248 [==============================] - 1s 5ms/step - loss: 0.6601 - accuracy: 0.7597 - val_loss: 0.6570 - val_accuracy: 0.7540
Epoch 19/50
248/248 [==============================] - 1s 5ms/step - loss: 0.6482 - accuracy: 0.7635 - val_loss: 0.6626 - val_accuracy: 0.7540
Epoch 00019: early stopping


Validation Accuracy Rate:,▁
Validation Error Rate:,▁
accuracy,▁▆▆▇▇▇▇▇███████████
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
loss,█▄▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
val_accuracy,▁▅▄▄▇▆▅▆▇▇▇███▇████
val_loss,█▄▅▅▂▃▃▃▂▁▁▁▁▁▂▁▂▁▁
Validation Accuracy Rate:,75.4
Validation Error Rate:,24.6
accuracy,0.76351
best_epoch,13
